In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('/content/train.csv')
test  = pd.read_csv('/content/test.csv')


# Reduce both datasets to 1000 entries each
train = train.sample(n=100, random_state=42).reset_index(drop=True)
test = test.sample(n=100, random_state=42).reset_index(drop=True)

print(train)


      id      keyword                     location  \
0   3796  destruction                          NaN   
1   3185       deluge                          NaN   
2   7769       police                           UK   
3    191   aftershock                          NaN   
4   9810       trauma        Montgomery County, MD   
..   ...          ...                          ...   
95  2038   casualties  50% Queanbeyan - 50% Sydney   
96  1526  body%20bags   Speaking the Truth in Love   
97   463   armageddon                          NaN   
98  4121      drought                Charlotte, NC   
99  9690      tornado                          NaN   

                                                 text  target  
0   So you have a new weapon that can cause un-ima...       1  
1   The f$&amp;@ing things I do for #GISHWHES Just...       0  
2   DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...       1  
3   Aftershock back to school kick off was great. ...       0  
4   in response to trauma Child

In [ ]:
# # Replace missing values in 'keyword' and 'location' with empty strings
# train['keyword'].fillna('', inplace=True)
# train['location'].fillna('', inplace=True)

# # Calculate the percentage of missing values after replacement
# missing_keyword = train['keyword'].isnull().mean() * 100
# missing_location = train['location'].isnull().mean() * 100

# print(f"Percentage of missing 'keyword': {missing_keyword}%")
# print(f"Percentage of missing 'location': {missing_location}%")

# # Check for any missing values in the 'keyword' and 'location' columns after replacing
# print("Missing values in 'keyword':", train['keyword'].isnull().sum())
# print("Missing values in 'location':", train['location'].isnull().sum())


In [ ]:
# Calculate percentage of missing values
missing_keyword = train['keyword'].isnull().mean() * 100
missing_location = train['location'].isnull().mean() * 100
print(f"Percentage of missing 'keyword': {missing_keyword}%")
print(f"Percentage of missing 'location': {missing_location}%")



print("Missing values in 'keyword':", train['keyword'].isnull().sum())
print("Missing values in 'location':", train['location'].isnull().sum())




Percentage of missing 'keyword': 1.0%
Percentage of missing 'location': 40.0%
Missing values in 'keyword': 1
Missing values in 'location': 40


In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords

# Download stopwords if you haven't
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove stopwords
    text = ' '.join(word for word in text.split() if word.lower() not in stop_words)

    # Remove emojis (using a Unicode range for emojis)
    text = re.sub(r'[\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U0001F1E0-\U0001F1FF]', '', text)

    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
train['cleaned_text'] = train['text'].apply(clean_text)

# Display the cleaned text
print(train[['text', 'cleaned_text']].head())

                                                text  \
0  So you have a new weapon that can cause un-ima...   
1  The f$&amp;@ing things I do for #GISHWHES Just...   
2  DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...   
3  Aftershock back to school kick off was great. ...   
4  in response to trauma Children of Addicts deve...   

                                        cleaned_text  
0          new weapon cause unimaginable destruction  
1  famping things GISHWHES got soaked deluge goin...  
2  DT georgegalloway RT Galloway4Mayor ÛÏThe CoL...  
3  Aftershock back school kick great want thank e...  
4  response trauma Children Addicts develop defen...  


In [ ]:
import pandas as pd
from transformers import BertTokenizer


# Load the pretrained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define a function to tokenize the text and convert to IDs
def bert_tokenize(text):
    # Tokenize the text using BERT's tokenizer
    tokens = tokenizer.tokenize(text)

    # Convert tokens to IDs
    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    return tokens, token_ids

# Apply the function to your dataset
train['bert_tokens'], train['bert_token_ids'] = zip(*train['text'].apply(bert_tokenize))

# Display the first few rows to check the output
print(train[['text', 'bert_tokens', 'bert_token_ids']].head())


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


                                                text  \
0  So you have a new weapon that can cause un-ima...   
1  The f$&amp;@ing things I do for #GISHWHES Just...   
2  DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...   
3  Aftershock back to school kick off was great. ...   
4  in response to trauma Children of Addicts deve...   

                                         bert_tokens  \
0  [so, you, have, a, new, weapon, that, can, cau...   
1  [the, f, $, &, amp, ;, @, ing, things, i, do, ...   
2  [dt, @, george, ##gall, ##owa, ##y, :, rt, @, ...   
3  [after, ##sho, ##ck, back, to, school, kick, o...   
4  [in, response, to, trauma, children, of, addic...   

                                      bert_token_ids  
0  [2061, 2017, 2031, 1037, 2047, 5195, 2008, 206...  
1  [1996, 1042, 1002, 1004, 23713, 1025, 1030, 13...  
2  [26718, 1030, 2577, 22263, 21293, 2100, 1024, ...  
3  [2044, 22231, 3600, 2067, 2000, 2082, 5926, 21...  
4  [1999, 3433, 2000, 12603, 2336, 1997, 26855, 2..

In [ ]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Define max sequence length
MAX_LEN = 64

# Pad the token IDs and create attention masks
input_ids = list(train['bert_token_ids'].apply(lambda x: x + [0] * (MAX_LEN - len(x)) if len(x) < MAX_LEN else x[:MAX_LEN]))
attention_masks = [[float(i > 0) for i in seq] for seq in input_ids]

# Convert lists to tensors
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)
labels = torch.tensor(train['target'].values)


In [ ]:
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, test_size=0.1, random_state=42)
train_masks, val_masks = train_test_split(attention_masks, test_size=0.1, random_state=42)

batch_size = 8

# Create DataLoader for training and validation
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)



In [ ]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_linear_schedule_with_warmup

# Set number of epochs
epochs = 2

# Total steps for the scheduler
total_steps = len(train_dataloader) * epochs

# Scheduler for learning rate decay
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # Training phase
    model.train()
    total_train_loss = 0

    for batch in train_dataloader:
        b_input_ids, b_input_mask, b_labels = batch

        # Clear gradients
        model.zero_grad()

        # Forward pass
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_train_loss += loss.item()

        # Backward pass
        loss.backward()

        # Clip gradients to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Update parameters
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"Average training loss: {avg_train_loss}")

    # Validation phase
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0

    for batch in val_dataloader:
        b_input_ids, b_input_mask, b_labels = batch

        # Turn off gradient calculations for validation
        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)

        loss = outputs.loss
        logits = outputs.logits

        total_eval_loss += loss.item()

        # Calculate accuracy
        preds = torch.argmax(logits, dim=1).flatten()
        accuracy = (preds == b_labels).cpu().numpy().mean()
        total_eval_accuracy += accuracy

    avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
    avg_val_loss = total_eval_loss / len(val_dataloader)

    print(f"Validation accuracy: {avg_val_accuracy}")
    print(f"Validation loss: {avg_val_loss}")


Epoch 1/2
Average training loss: 0.6641874810059866
Validation accuracy: 0.375
Validation loss: 0.7677536010742188
Epoch 2/2
Average training loss: 0.6283511320749918
Validation accuracy: 0.375
Validation loss: 0.7880755066871643


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
from google.colab import files
import shutil

# Initialize model and tokenizer
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Save model and tokenizer in Colab
save_directory = "/content/tweetModel"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

# Zip the folder containing the model and tokenizer
shutil.make_archive(save_directory, 'zip', save_directory)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


'/content/tweetModel.zip'

In [ ]:
# Function to preprocess and predict tweet sentiment
def predict_tweet_sentiment(tweet):
    # Tokenize the input tweet
    inputs = tokenizer(tweet, return_tensors="pt", truncation=True, padding=True, max_length=64)

    # Move inputs to GPU if available (optional)
    if torch.cuda.is_available():
        inputs = {key: value.cuda() for key, value in inputs.items()}

    # Model prediction
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Get the predicted class (0 or 1)
    prediction = torch.argmax(logits, dim=1).item()

    # Return the prediction
    return "Disaster" if prediction == 1 else "Not Disaster"


In [ ]:
# Test the function by entering a tweet
tweet = input("Enter a tweet to classify: ")
prediction = predict_tweet_sentiment(tweet)
print(f"Prediction: {prediction}")


In [ ]:
from sklearn.metrics import f1_score

# Validation phase
model.eval()
total_eval_accuracy = 0
total_eval_loss = 0
all_preds = []
all_labels = []

for batch in val_dataloader:
    b_input_ids, b_input_mask, b_labels = batch

    # Turn off gradient calculations for validation
    with torch.no_grad():
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)

    loss = outputs.loss
    logits = outputs.logits

    total_eval_loss += loss.item()

    # Calculate accuracy
    preds = torch.argmax(logits, dim=1).flatten()
    accuracy = (preds == b_labels).cpu().numpy().mean()
    total_eval_accuracy += accuracy

    # Collect all predictions and labels for F1 score calculation
    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(b_labels.cpu().numpy())

# Calculate F1 score
f1 = f1_score(all_labels, all_preds, average='weighted')  # or use 'macro' for macro-average
avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
avg_val_loss = total_eval_loss / len(val_dataloader)

print(f"Validation accuracy: {avg_val_accuracy}")
print(f"Validation loss: {avg_val_loss}")
print(f"Validation F1 score: {f1}")
